# Лабораторная работа # 1

# 0. Инициализация

In [ ]:
# Для анимаций
from matplotlib import rc
rc('animation', html='jshtml')
from matplotlib.animation import FuncAnimation

# Для больших анимаций
import matplotlib as mpl
mpl.rcParams['animation.embed_limit'] = 1000.0 

# Для графиков и прочего
import numpy as np
import scipy
from matplotlib import cm
import matplotlib.pyplot as plt
import math
import pandas as pd

# Для корректного отображения 
%matplotlib inline
plt.style.use('fivethirtyeight')

# Для таблиц
import csv 

# Для работы с файлами
import os

# Для progress bar
from tqdm import tqdm

In [ ]:
root_path = 'source' + os.sep

## Градиент

In [ ]:
delta = 1e-9
def grad(f, x):
    """
    Функция вычисления градиента в заданной точке с константной точностью

    Аргументы:
    f -- функция
    x -- точка

    Возвращает:
    ans -- градиент функции в точке x
    """

    n = len(x)
    xd = np.copy(x)
    ans = np.zeros(n)

    for i in range(n):
        xd[i] += delta
        ans[i] = np.divide(f(xd) - f(x), delta)
        xd[i] -= delta

    return ans

## Градиентный спуск

In [ ]:
def gradient_descent(f, x0, lr_search_func, num_iterations=1000, eps=1e-6, step_size=1, minimum=0, apply_min=False):
    """
    Функция вычисления градиентного спуска с заданной функцией поиска коэффициента обучения

    Аргументы:
    f -- функция
    x0 -- начальная точка
    ----------------------------------------------------------------------------
    lr_search_func -- функция поиска оптимального коэффициента обучения (learning rate)
        Аргументы:
        f -- функция
        a -- левая граница интервала
        b -- правая граница интервала
        eps -- точность поиска

        Возвращает:
        x -- точка минимума функции
    ----------------------------------------------------------------------------
    eps -- точность поиска
    num_iterations -- количество итераций
    step_size -- размер шага

    Возвращает:
    points -- массив оптимальных на каждом шаге точек
    """
    def line_search(x, d):
        fd = lambda alpha: f(x - alpha*d)
        alpha = lr_search_func(fd, 0, 1, eps)
        return alpha

    x = np.copy(x0)
    points = np.array([x])
    for i in range(num_iterations):
        if apply_min and abs(f(x) - minimum) < eps:
            break

        grad_x = grad(f, x);
        x = x - grad_x * line_search(x, grad_x) * step_size
        points = np.vstack([points, x])
    return points

## Для отрисовки

In [ ]:
def init():
    global X, Y, Z, f, x0
    Z = np.vectorize(lambda x, y: f(np.array([x, y])))(X, Y)

### 3D визуализация функции

In [ ]:
def print_f(elev=30, azim=60):
  # Создание фигуры и трехмерной оси
  fig = plt.figure(figsize=(10, 10))
  ax = fig.add_subplot(111, projection='3d')

  # Установка угол обзора
  ax.view_init(elev=elev, azim=azim)

  # Построение поверхности
  ax.plot_surface(X, Y, Z)

  # Построение начальной точки
  ax.plot(x0[0], x0[1], f(x0), 'ro', label='Начальная точка')

  # Установка отступа между графиком и значениями осей
  ax.tick_params(pad=10)

  # Добавление легенды
  plt.legend(loc='upper left')

  # Установка размера шрифта для подписей осей
  ax.tick_params(axis='x', labelsize=10)
  ax.tick_params(axis='y', labelsize=10)
  ax.tick_params(axis='z', labelsize=10)

  # Добавление заголовка и подписей осей
  plt.title('График функции с начальной точкой')
  ax.set_xlabel('Ось X', labelpad=20.0)
  ax.set_ylabel('Ось Y', labelpad=20.0)
  ax.set_zlabel('Ось f(x, y)', labelpad=20.0)

  # Отображение графика
  plt.show()

In [ ]:
def print_f_animated(interval=100, elev = 30, st_azim = 80, delta=5):
    # plt.title('График функции с начальной точкой')
    fig = plt.figure(figsize=(7, 7))
    ax = plt.axes(projection='3d')
    def frame(w):
        ax.clear()

        # угол обзора
        azim = (w*delta + st_azim)%360
        ax.view_init(elev=elev, azim=azim)

        # Построение поверхности
        ax.plot_surface(X, Y, Z)

        # Построение начальной точки
        label = 'elev=' + str(elev) + ', azim=' + str(azim)
        ax.plot(x0[0], x0[1], f(x0), 'ro', markersize=3, label=label)

        # Установка отступа между графиком и значениями осей
        ax.tick_params(pad=10)

        # Установка размера шрифта для подписей осей
        ax.tick_params(axis='x', labelsize=10)
        ax.tick_params(axis='y', labelsize=10)
        ax.tick_params(axis='z', labelsize=10)
        
        # Добавление заголовка и подписей осей
        ax.set_xlabel('Ось X', labelpad=20.0)
        ax.set_ylabel('Ось Y', labelpad=20.0)
        ax.set_zlabel('Ось f(x, y)', labelpad=20.0)

        ax.legend(loc='upper left')

        return ax
    plt.close()

    frames = np.ceil(360 / delta).astype(int)

    return FuncAnimation(fig, frame, interval=interval, frames=frames, blit=False, repeat=True)

### Линии уровня и градиент

In [ ]:
def print_lines_grad(list_result, list_label, title='Градиентный спуск на уровнях функции', filename='', filename_extension='.png', dpi=1024):
  fig = plt.figure(figsize=(10, 10))
  ax = fig.add_subplot(111)

  for i in range(len(list_result)):
    levels=np.unique(np.sort(f(list_result[i])))
    ax.contour(X, Y, Z, levels=levels, colors='red', antialiased=True, linewidths=1.0)

  for i in range(len(list_result)):
    x = list_result[i][:, 0]
    y = list_result[i][:, 1]
    ax.plot(x, y, marker='.', markersize=10, markerfacecolor='black', label=list_label[i], linewidth = 2)
    print(f'{list_label[i]:15} ==> {f(list_result[i][-1]):10f} in [{list_result[i][-1][0]:10f}, {list_result[i][-1][1]:10f}]')
    
  # Добавление легенды
  if len(list_label) > 0:
    plt.legend(loc='upper left')

  if(filename != ''):
    plt.savefig(filename + filename_extension, dpi=dpi, bbox_inches=0, transparent=True)    

  plt.show()

In [ ]:
def print_lines_grad_animated(list_result, list_label, interval=100, frames=-1):
    fig = plt.figure(figsize=(7, 7))
    ax = fig.add_subplot(111)
    def frame(w):
        ax.clear()

        for i in range(len(list_result)):
          levels=np.unique(np.sort(f(list_result[i][:frames].T)))
          ax.contour(X, Y, Z, levels, colors='red', antialiased=True, linewidths=1.0)

        for i in range(len(list_result)):
          x = list_result[i][:w, 0]
          y = list_result[i][:w, 1]
          ax.plot(x, y, marker='.', markersize=10, markerfacecolor='black', label=list_label[i], linewidth = 2)

        ax.legend(loc='upper left')

        return ax

    plt.close()
    if frames == -1 or frames > len(list_result[0]):
      frames = len(list_result[0])

    return FuncAnimation(fig, frame, interval=interval, frames=frames, blit=False, repeat=True)

### 3D и градиент

In [ ]:
def print_full_grad(list_result, list_label, title='Градиентный спуск на графике функции', elev = 30, azim = 80, filename='', filename_extension='.png', dpi=1024):
  fig = plt.figure(figsize=(10, 10))
  ax = fig.add_subplot(projection='3d')

  for i in range(len(list_result)):
    x = list_result[i][:, 0]
    y = list_result[i][:, 1]
    z = np.vectorize(lambda x, y: f(np.array([x, y])))(x, y)
    ax.plot(x, y, marker='.', markersize=10, markerfacecolor='black', zs=z, label=list_label[i], linewidth = 2)
    print(f'{list_label[i]:15} ==> {f(list_result[i][-1]):10f} in [{list_result[i][-1][0]:10f}, {list_result[i][-1][1]:10f}]')
    
  ax.plot_surface(X, Y, Z, cmap=cm.coolwarm)
  ax.view_init(elev=elev, azim=azim)
  
  # Установка отступа между графиком и значениями осей
  ax.tick_params(pad=10)

  # Добавление легенды
  if len(list_label) > 0:
    ax.legend(loc='upper left')

  # Установка размера шрифта для подписей осей
  ax.tick_params(axis='x', labelsize=10)
  ax.tick_params(axis='y', labelsize=10)
  ax.tick_params(axis='z', labelsize=10)

  # Добавление заголовка и подписей осей
  if title != '':
    plt.title(title)
  
  ax.set_xlabel('Ось X', labelpad=20.0)
  ax.set_ylabel('Ось Y', labelpad=20.0)
  ax.set_zlabel('Ось f(x, y)', labelpad=20.0)


  if(filename != ''):
    plt.savefig(filename + filename_extension, dpi=dpi, bbox_inches=0, transparent=True)

  plt.show()


In [ ]:
def print_full_grad_animated(list_result, list_label, interval=100, frames=-1, elev = 30, azim = 80):
    fig = plt.figure(figsize=(7, 7))
    ax = plt.axes(projection='3d')

    def frame(w):
        ax.clear()
        for i in range(len(list_result)):
            x = list_result[i][:w+1, 0]
            y = list_result[i][:w+1, 1]
            z = np.vectorize(lambda x, y: f(np.array([x, y])))(x, y)
            ax.plot(x, y, marker='.', markersize=10, markerfacecolor='black', zs=z, label=list_label[i], linewidth = 2, markevery=(w,w+1))

        ax.plot_surface(X, Y, Z, cmap=cm.coolwarm)
        ax.view_init(elev=elev, azim=azim)
        if len(list_label) > 0:
            ax.legend(loc='upper left')

        return ax

    plt.close()
    if frames == -1 or frames > len(list_result[0]):
      frames = len(list_result[0])

    return FuncAnimation(fig, frame, interval=interval, frames=frames, blit=False, repeat=True)

## Для вывода

In [ ]:
def print_result(result, sep=''):
    global f
    print(len(result), ":")
    for i in range(len(result)):
        print(i, sep, result[i], sep, f(result[i]))

In [ ]:
def save_result_text(result, filename, sep=' ', sp='%g'):
    with open(filename, 'w') as file:
        for data in result:
            for x in data:
                file.write((sp + "%s") % (x, sep))  
            file.write((sp + "\n") % f(data))

In [ ]:
def save_result_table(result, filename, sp='%g', fields=[], generate_fields=False):
    with open(filename, 'w') as csvfile:  
        # создание объекта witer csv
        csvwriter = csv.writer(csvfile, quoting=csv.QUOTE_NONE)  
            
        if generate_fields:
            if len(result[0]) == 1:
                fields = ['X']
            elif len(result[0]) == 2:
                fields = ['X', 'Y']
            else:
                fields = [f"X[{i}]" for i in range(len(result[0]))]
            fields.append('F')

        # запись шапки
        if len(fields) > 0:
            csvwriter.writerow(fields)  
            
        # запись данных 
        data = np.insert(result, len(result[0]), [f(x) for x in result], axis=1)
        formatted_data = [[sp % x for x in row] for row in data]
        csvwriter.writerows(formatted_data)

In [ ]:
def save_result(list_result, list_label, filepath='', sp='%g', fields=[], generate_fields=True):
    for i in range(len(list_result)):
        save_result_text(list_result[i], filepath + list_label[i] + '.txt', sp=sp)
        save_result_table(list_result[i], filepath + list_label[i] + '.csv', sp, fields, generate_fields)

# 1. Реализуйте градиентный спуск с постоянным шагом (learning rate).

In [ ]:
def gradient_descent_constant(f, x0, lr=0.01, num_iterations=1000, minimum=0, apply_min=False):
    """
    Градиентный спуск с постоянным шагом.

    Аргументы:
    f -- функция
    x0 -- начальная точка
    lr -- постоянный коэффициент обучения (learning rate)
    num_iterations -- количество итераций

    Возвращает:
    gradient_descent(...)
    """
    def const_lr(f, a, b, eps=1e-6):
        return lr
    
    return gradient_descent(f, x0, const_lr, num_iterations, minimum=minimum, apply_min=apply_min)

In [ ]:
filepath = root_path + 'Task1' + os.sep

## Пример 1:

In [ ]:
fileprefix = 'f1_'

def f(x):
    return x[0] ** 2 + x[1] ** 2

x = np.linspace(-50, 50, 120)
y = np.linspace(-100, 100, 120)
X, Y = np.meshgrid(x, y)
x0 = np.array([25, -50], dtype=float)
init()

num_iter = 30;
result = [gradient_descent_constant(f, x0, lr=0.2, num_iterations=num_iter)]
result_label = ['constant']

# print_result(result[0])
save_result(result, result_label, filepath + fileprefix)
# print_full_grad(result, result_label, title='Пример 1: градиентный спуск на графике функции', azim=10, filename=filepath + fileprefix + '_'.join(result_label))
print_full_grad(result, result_label, title='', azim=10, filename=filepath + fileprefix + '_'.join(result_label))

## Пример 2:

In [ ]:
fileprefix = 'f2_'

def f(x):
    return -0.84233647 * x[0] ** 2 + -0.28077882 * x[1] ** 2

x = np.linspace(-50, 50, 120)
y = np.linspace(-100, 100, 120)
X, Y = np.meshgrid(x, y)
x0 = np.array([0.00000014, 0.1], dtype=float)
init()

num_iter = 126
result = [gradient_descent_constant(f, x0, lr=0.1, num_iterations=num_iter)]
result_label = ['constant']

# print_result(result[0])
save_result(result, result_label, filepath + fileprefix)
# print_full_grad(result, result_label, title='Пример 2: градиентный спуск на графике функции', azim=10, filename=filepath + fileprefix + '_'.join(result_label))
print_full_grad(result, result_label, title='', azim=10, filename=filepath + fileprefix + '_'.join(result_label))

## Пример 3:

In [ ]:
fileprefix = 'f3_'

def f(x):
    return (x[0]**2 - x[1]**2 - 9)*np.cos(2*x[0]+1-np.exp(x[1]))

x = np.linspace(-3, 3, 120)
y = np.linspace(-3, 1, 120)
X, Y = np.meshgrid(x, y)
x0 = np.array([-0.1, 0.4], dtype=float)
init()

num_iter = 50;
result = [gradient_descent_constant(f, x0, lr=0.07, num_iterations=num_iter)]
result_label = ['constant']

# print_result(result[0])
save_result(result, result_label, filepath + fileprefix)
# print_full_grad(result, result_label, title='Пример 3: градиентный спуск на графике функции', elev=45, azim=75, filename=filepath + fileprefix + '_'.join(result_label))
print_full_grad(result, result_label, title='', elev=45, azim=75, filename=filepath + fileprefix + '_'.join(result_label))

# 2. Реализуйте метод одномерного поиска (метод дихотомии, метод Фибоначчи, метод золотого сечения) и градиентный спуск на его основе.

## (a) Метод дихотомии

In [ ]:
def dichotomy_search(f, a, b, eps=1e-6):
    """
    Метод дихотомии для поиска минимума функции f на интервале [a,b] с точностью eps

    Аргументы:
    f -- функция
    a -- начальная точка интервала
    b -- конечная точка интервала
    eps -- точность поиска

    Возвращает:
    x -- точка минимума функции
    """
    while b - a > eps:
        c = (a + b) / 2
        if f(c - eps) < f(c + eps):
            b = c
        else:
            a = c
    return (a + b) / 2

def gradient_descent_dichotomy(f, x0, num_iterations=1000, eps=1e-6, step_size=0.01, minimum=0, apply_min=False):
    """
    Градиентный спуск на основе метода дихотомии

    Аргументы:
    f -- функция
    x0 -- начальная точка
    num_iterations -- количество итераций
    eps -- точность поиска
    step_size -- размер шага
    
    Возвращает:
    gradient_descent(...)
    """

    return gradient_descent(f, x0, dichotomy_search, num_iterations, eps, step_size, minimum=minimum, apply_min=apply_min)

## (b) Метод Фибоначчи

In [ ]:
def fibonacci_search(f, a, b, n):
    """
    Метод Фибоначчи для одномерного поиска.

    Аргументы:
    f -- функция
    a -- левая граница интервала
    b -- правая граница интервала
    n -- "точность" поиска функции

    Возвращает:
    x -- точка минимума
    """

    fib = [1, 1]
    while fib[-1] < n:
        fib.append(fib[-1] + fib[-2])

    k = len(fib) - 1
    x1 = a + (fib[k - 2] / fib[k]) * (b - a)
    x2 = a + (fib[k - 1] / fib[k]) * (b - a)
    f1 = f(x1)
    f2 = f(x2)

    for i in range(k - 2):
        if f1 < f2:
            b = x2
            x2 = x1
            f2 = f1
            x1 = a + (fib[k - i - 3] / fib[k - i - 1]) * (b - a)
            f1 = f(x1)
        else:
            a = x1
            x1 = x2
            f1 = f2
            x2 = a + (fib[k - i - 2] / fib[k - i - 1]) * (b - a)
            f2 = f(x2)

    return (a + b) / 2

def gradient_descent_fibonacci(f, x0, num_iterations=1000, n=6900, step_size=0.01, minimum=0, apply_min=False):
    """
    Градиентный спуск на основе метода Фибоначчи

    Аргументы:
    f -- функция
    x0 -- начальная точка
    num_iterations -- количество итераций
    n -- "точность" поиска
    step_size -- размер шага
    
    Возвращает:
    gradient_descent(...)
    """

    return gradient_descent(f, x0, fibonacci_search, num_iterations, n, step_size, minimum=minimum, apply_min=apply_min)

## (c) Метод золотого сечения

In [ ]:
def golden_section_search(f, a, b, eps=1e-6):
    """
    Метод золотого сечения для одномерного поиска.

    Аргументы:
    f -- функция
    a -- левая граница интервала
    b -- правая граница интервала
    eps -- точность поиска функции

    Возвращает:
    x -- точка минимума
    """
    phi = (1 + np.sqrt(5)) / 2
    x1 = b - (b - a) / phi
    x2 = a + (b - a) / phi
    while abs(b - a) > eps:
        if f(x1) < f(x2):
            b = x2
        else:
            a = x1
        x1 = b - (b - a) / phi
        x2 = a + (b - a) / phi
    return (a + b) / 2

def gradient_descent_golden_section(f, x0, num_iterations=1000, eps=1e-6, step_size=0.01, minimum=0, apply_min=False):
    """
    Градиентный спуск на основе метода золотого сечения.

    Аргументы:
    f -- функция
    x0 -- начальная точка
    num_iterations -- количество итераций
    eps -- точность поиска

    Возвращает:
    gradient_descent(...)
    """
    return gradient_descent(f, x0, golden_section_search, num_iterations, eps, step_size, minimum=minimum, apply_min=apply_min)

In [ ]:
filepath = root_path + 'Task2' + os.sep

## Пример 1:

In [ ]:
fileprefix = 'f1_'

# Функция Розенброка min => f(1, 1) = 0
f = scipy.optimize.rosen

x = np.linspace(-1, 1.5, 120)
y = np.linspace(-1, 2, 120)
X, Y = np.meshgrid(x, y)
x0 = np.array([0.4, -0.9], dtype=float)
init()

num_iter = 10;
result = [gradient_descent_constant(f, x0, lr=.005, num_iterations=num_iter),
          gradient_descent_dichotomy(f, x0, step_size=1, num_iterations=num_iter)]
result_label = ['constant', 'dichotomy']

save_result(result, result_label, filepath + fileprefix)
print_full_grad(result, result_label, title='', elev=25, azim=100, filename=filepath + fileprefix + '_'.join(result_label))

## Пример 2:

In [ ]:
fileprefix = 'f2_'

# Функция Гольдшейна-Прайса min => f(0, -1) = 3
def f(x):
    return (1 + (x[0] + x[1] + 1)**2 * (19 - 14*x[0] + 3*x[0]**2-14*x[1]+6*x[0]*x[1]+3*x[1]**2)) * (30 + (2*x[0] - 3*x[1])**2 * (18 - 32*x[0] + 12*x[0]**2 + 48*x[1] - 36*x[0]*x[1]+27*x[1]**2))

x = np.linspace(-2, 1, 120)
y = np.linspace(-2, 1, 120)
X, Y = np.meshgrid(x, y)
x0 = np.array([-0.9, 0.7], dtype=float)
init()

num_iter = 7;
result = [gradient_descent_constant(f, x0, lr=.000005, num_iterations=num_iter),
          gradient_descent_dichotomy(f, x0, step_size=0.8, num_iterations=num_iter)]
result_label = ['constant', 'dichotomy']

save_result(result, result_label, filepath + fileprefix)
print_full_grad(result, result_label, title='', elev=25, azim=10, filename=filepath + fileprefix + '_'.join(result_label))

# 3. Проанализируйте траекторию градиентного спуска на примере квадратичных функций. Для этого придумайте две-три квадратичные функции от двух переменных, на которых работа методов будет отличаться.

In [ ]:
filepath = root_path + 'Task3' + os.sep

## Пример 1:

In [ ]:
fileprefix = 'f1_'

def f(x):
    return x[0] ** 2 + x[1] ** 2

x = np.linspace(-50, 50, 120)
y = np.linspace(-75, 75, 120)
X, Y = np.meshgrid(x, y)
x0 = np.array([35, 50], dtype=float)

init()


num_iter = 60
result = [gradient_descent_constant(f, x0, lr=1, num_iterations=num_iter),
          gradient_descent_dichotomy(f, x0, step_size=1, num_iterations=num_iter)]
result_label = ['constant', 'dichotomy']

save_result(result, result_label, filepath + fileprefix)
print_full_grad(result, result_label, title='', elev=30, azim=-14, filename=filepath + fileprefix + '_'.join(result_label))

## Пример 2:

In [ ]:
fileprefix = 'f2_'

def f(x):
	A = np.array([[0.7464451039232642, 1.0905399322509766], [-0.45261597812262555, 0.5415655721418664]])
	return (x.dot(A)).dot(x.T)

x = np.linspace(-50, 50, 120)
y = np.linspace(-75, 75, 120)
X, Y = np.meshgrid(x, y)
x0 = np.array([50, -75], dtype=float)

init()


num_iter = 100
result = [gradient_descent_constant(f, x0, lr=0.2, num_iterations=num_iter),
          gradient_descent_dichotomy(f, x0, step_size=1, num_iterations=num_iter)]
result_label = ['constant', 'dichotomy']

save_result(result, result_label, filepath + fileprefix)
print_full_grad(result, result_label, title='', elev=30, azim=-18, filename=filepath + fileprefix + '_'.join(result_label))

# 4. Для каждой функции:


In [ ]:
filepath = root_path + 'Task4' + os.sep

## (a) исследуйте сходимость градиентного спуска с постоянным шагом, сравните полученные результаты для выбранных функций;

In [ ]:
def gradient_descent_constant_with_end_condition(x0, lr, eps, max_iter, minimum):
    x = np.copy(x0)
    steps = 0

    for i in range(max_iter):      
        if abs(f(x) - minimum) < eps:
            break

        steps += 1
        grad_res = grad(f, x)
        x = x - lr * grad_res

    return steps

def constant_research_lr(minimum, lr_step_size=1e-04, max_iter=1000, eps=1e-6, filename='', filename_extension='.png', dpi=1024):
    lr_values = np.arange(0, 1, lr_step_size)

    result = []

    for lr_iter in tqdm(lr_values):
        steps = gradient_descent_constant_with_end_condition(x0, lr_iter, eps, max_iter, minimum)
        result.append(steps)

    min_lr_index = np.argmin(result)
    min_steps = result[min_lr_index]
    min_lr = lr_values[min_lr_index]

    plt.annotate(f'[{min_lr}, {min_steps}]', 
                xy=(min_lr, min_steps),
                xytext=(min_lr-0.1, min_steps+max_iter/10),
                bbox=dict(boxstyle="round", fc="0.8"),
                arrowprops=dict(arrowstyle='->', lw=2, color='black'))
    
    plt.xlabel('Learning rate', fontsize=14)
    plt.ylabel('Steps', fontsize=14)
    plt.plot(lr_values, result)
    
    if(filename != ''):
        plt.savefig(filename + filename_extension, dpi=dpi, bbox_inches=0, transparent=True)

    plt.show()

### Пример 1: 

In [ ]:
fileprefix = 'a_f1_'

def f(x):
    return x[0] ** 2 + x[1] ** 2

x = np.linspace(-50, 50, 120)
y = np.linspace(-75, 75, 120)
X, Y = np.meshgrid(x, y)
x0 = np.array([35, 50], dtype=float)

init()
constant_research_lr(0, 1e-3, max_iter=10000, filename=filepath + fileprefix + 'constant_research_lr')

### Пример 2: 

In [ ]:
fileprefix = 'a_f2_'

def f(x):
	A = np.array([[0.7464451039232642, 1.0905399322509766], [-0.45261597812262555, 0.5415655721418664]])
	return (x.dot(A)).dot(x.T)

x = np.linspace(-50, 50, 120)
y = np.linspace(-75, 75, 120)
X, Y = np.meshgrid(x, y)
x0 = np.array([50, -75], dtype=float)

init()
constant_research_lr(0, 1e-3, max_iter=1000, filename=filepath + fileprefix + 'constant_research_lr')

## (b) сравните эффективность градиентного спуска с использованием одномерного поиска с точки зрения количества вычислений минимизируемой функции и ее градиентов;

In [ ]:
def line_search_and_constant_research_step_size(minimum, lr_step_size=1e-04, max_iter=1000, eps=1e-6, filename='', filename_extension='.png', dpi=1024):
    step_size_values = np.arange(lr_step_size, 1, lr_step_size)

    table = {'constant': [],
            'dichotomy': []}
    
    global counter
    for lr_iter in tqdm(step_size_values):
        counter = 0
        gradient_descent_constant(f, x0, lr=lr_iter, num_iterations=max_iter, minimum=minimum, apply_min=True)
        table['constant'].append(counter)

        counter = 0
        gradient_descent_dichotomy(f, x0, step_size=lr_iter, num_iterations=max_iter, eps=eps, minimum=minimum, apply_min=True)
        table['dichotomy'].append(counter)

    min_constant_f_count_index = np.argmin(table['constant'])
    min_dichotomy_f_count_index = np.argmin(table['dichotomy'])

    plt.xlabel('Step size', fontsize=14)
    plt.ylabel('F(x) count', fontsize=14)
    plt.plot(step_size_values, table['constant'], label='constant: min - ' + str(table['constant'][min_constant_f_count_index]) + ' in ' + '%g' % step_size_values[min_constant_f_count_index])
    plt.plot(step_size_values, table['dichotomy'], label='dichotomy: min - ' + str(table['dichotomy'][min_dichotomy_f_count_index]) + ' in ' + '%g' % step_size_values[min_dichotomy_f_count_index])
    plt.legend(loc='upper left')

    if(filename != ''):
        plt.savefig(filename + filename_extension, dpi=dpi, bbox_inches=0, transparent=True)

    plt.show()

### Пример 1: 

In [ ]:
fileprefix = 'b_f1_'

counter = 0
def f(x):
    global counter
    counter += 1
    return x[0] ** 2 + x[1] ** 2

x = np.linspace(-50, 50, 120)
y = np.linspace(-75, 75, 120)
X, Y = np.meshgrid(x, y)
x0 = np.array([35, 50], dtype=float)

init()
line_search_and_constant_research_step_size(0, 1e-3, max_iter=1000, filename=filepath + fileprefix + 'line_search_and_constant_research_step_size')

### Пример 2: 

In [ ]:
fileprefix = 'b_f2_'

counter = 0
def f(x):
    global counter
    counter += 1
    A = np.array([[0.7464451039232642, 1.0905399322509766], [-0.45261597812262555, 0.5415655721418664]])
    return (x.dot(A)).dot(x.T)

x = np.linspace(-50, 50, 120)
y = np.linspace(-75, 75, 120)
X, Y = np.meshgrid(x, y)
x0 = np.array([50, -75], dtype=float)

init()
line_search_and_constant_research_step_size(0, 1e-3, max_iter=1000, filename=filepath + fileprefix + 'line_search_and_constant_research_step_size')

## (c) исследуйте работу методов в зависимости от выбора начальной точки;

In [ ]:
def get_count_step_constant(x, y):
    global f
    global x0
    x0 = np.array([x, y])
    return len(gradient_descent_constant(f, x0, lr=0.05, num_iterations=1000, minimum=0, apply_min=True))

def get_count_step_dichotomy(x, y):
    global f
    global x0
    x0 = np.array([x, y])
    return len(gradient_descent_dichotomy(f, x0, step_size=0.5, num_iterations=1000, minimum=0, apply_min=True))

### Пример 1:

In [ ]:
fileprefix = 'c_f1_'

def f(x):
    return x[0] ** 2 + x[1] ** 2

x = np.linspace(-75, 75, 120)
y = np.linspace(-75, 75, 120)
X, Y = np.meshgrid(x, y)

Z = np.vectorize(get_count_step_constant)(X, Y)
print_full_grad([], [], title='', elev=30, azim=-14, filename=filepath + fileprefix + 'constant')
Z = np.vectorize(get_count_step_dichotomy)(X, Y)
print_full_grad([], [], title='', elev=30, azim=-14, filename=filepath + fileprefix + 'dichotomy')

### Пример 2:

In [ ]:
fileprefix = 'c_f2_'

def f(x):
	A = np.array([[0.7464451039232642, 1.0905399322509766], [-0.45261597812262555, 0.5415655721418664]])
	return (x.dot(A)).dot(x.T)

x = np.linspace(-50, 50, 20)
y = np.linspace(-75, 75, 20)
X, Y = np.meshgrid(x, y)

Z = np.vectorize(get_count_step_constant)(X, Y)
print_full_grad([], [], title='', elev=30, azim=-14, filename=filepath + fileprefix + 'constant')
Z = np.vectorize(get_count_step_dichotomy)(X, Y)
print_full_grad([], [], title='', elev=30, azim=-14, filename=filepath + fileprefix + 'dichotomy')

## (e) в каждом случае нарисуйте графики с линиями уровня и траекториями методов;

### Пример 1:

In [ ]:
fileprefix = 'e_f1_'

def f(x):
    return x[0] ** 2 + x[1] ** 2

x = np.linspace(-50, 50, 120)
y = np.linspace(-75, 75, 120)
X, Y = np.meshgrid(x, y)
x0 = np.array([35, 50], dtype=float)

init()


num_iter = 60
result = [gradient_descent_constant(f, x0, lr=1, num_iterations=num_iter),
          gradient_descent_dichotomy(f, x0, step_size=1, num_iterations=num_iter)]
result_label = ['constant', 'dichotomy']

print_lines_grad(result, result_label, title='', filename=filepath + fileprefix + '_'.join(result_label))

### Пример 2:

In [ ]:
fileprefix = 'e_f2_'

def f(x):
	A = np.array([[0.7464451039232642, 1.0905399322509766], [-0.45261597812262555, 0.5415655721418664]])
	return (x.dot(A)).dot(x.T)

x = np.linspace(-50, 50, 120)
y = np.linspace(-75, 75, 120)
X, Y = np.meshgrid(x, y)
x0 = np.array([50, -75], dtype=float)

init()


num_iter = 100
result = [gradient_descent_constant(f, x0, lr=0.2, num_iterations=num_iter),
          gradient_descent_dichotomy(f, x0, step_size=1, num_iterations=num_iter)]
result_label = ['constant', 'dichotomy']

print_lines_grad(result, result_label, title='', filename=filepath + fileprefix + '_'.join(result_label))

# 5. Реализуйте генератор случайных квадратичных функций $n$ переменных с числом обусловленности $k$.

# Test

In [ ]:
def f(x):
    return x[0] ** 2 + x[1] ** 2

x = np.linspace(-50, 50, 120)
y = np.linspace(-100, 100, 120)
X, Y = np.meshgrid(x, y)
x0 = np.array([25, -50], dtype=float)

init()

print_f(azim=20)
# print_f_animated(delta=1)

In [ ]:
num_iter = 1000;

result = [gradient_descent_constant(f, x0, lr=.5, num_iterations=num_iter),
          gradient_descent_dichotomy(f, x0, step_size=1, num_iterations=num_iter),
          gradient_descent_golden_section(f, x0, step_size=1, num_iterations=num_iter),
          gradient_descent_fibonacci(f, x0, step_size=1, num_iterations=num_iter)]
result_label = ['constant', 'dichotomy', 'golden_section', 'fibonacci']

In [ ]:
print_full_grad(result, result_label, azim=10)
# print_full_grad_animated(result, result_label, interval=100, frames=100, azim=200)

In [ ]:
print_lines_grad(result, result_label)
# print_lines_grad_animated(result, result_label, interval=100, frames=100)